In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv(r'/content/drive/MyDrive/Data sets/complaints_processed.csv')
df

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...
...,...,...,...
162416,162416,debt_collection,name
162417,162417,credit_card,name
162418,162418,debt_collection,name
162419,162419,credit_card,name


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162421 entries, 0 to 162420
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  162421 non-null  int64 
 1   product     162421 non-null  object
 2   narrative   162411 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


In [ ]:
df.dropna(axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162411 entries, 0 to 162420
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  162411 non-null  int64 
 1   product     162411 non-null  object
 2   narrative   162411 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.0+ MB


In [ ]:
df

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...
...,...,...,...
162416,162416,debt_collection,name
162417,162417,credit_card,name
162418,162418,debt_collection,name
162419,162419,credit_card,name


In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
...,...,...
162416,debt_collection,name
162417,credit_card,name
162418,debt_collection,name
162419,credit_card,name


In [ ]:
empty_idx=[]
for indx,prd,txt in df.itertuples():
    if txt.isspace():
      empty_idx.append(indx)
    elif txt=='name':
      empty_idx.append(indx)
print(empty_idx)

[162415, 162416, 162417, 162418, 162419, 162420]


In [ ]:
df.drop(empty_idx,axis=0,inplace=True)
df

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
...,...,...
162410,credit_reporting,zales comenity bank closed sold account report...
162411,retail_banking,zelle suspended account without cause banking ...
162412,debt_collection,zero contact made debt supposedly resolved fou...
162413,mortgages_and_loans,zillow home loan nmls nmls actual quote provid...


In [ ]:
df['product'].value_counts()

credit_reporting       91171
debt_collection        23145
mortgages_and_loans    18990
credit_card            15564
retail_banking         13535
Name: product, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df['product']=le.fit_transform(df['product'])
df

,product,narrative
0,0,purchase order day shipping amount receive pro...
1,0,forwarded message date tue subject please inve...
2,4,forwarded message cc sent friday pdt subject f...
3,1,payment history missing credit report speciali...
4,1,payment history missing credit report made mis...
...,...,...
162410,1,zales comenity bank closed sold account report...
162411,4,zelle suspended account without cause banking ...
162412,2,zero contact made debt supposedly resolved fou...
162413,3,zillow home loan nmls nmls actual quote provid...


In [ ]:
encoded_values = [0, 1, 2, 3, 4]
original_labels = le.inverse_transform(encoded_values)
i=0
for label in original_labels:
    print(f"{encoded_values[i]} = {label}")
    i+=1

0 = credit_card
1 = credit_reporting
2 = debt_collection
3 = mortgages_and_loans
4 = retail_banking


In [ ]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken=[t for t in tokens if (t.isalpha())]
  stop=stopwords.words('english')
  ctoken=[t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return ' '.join(ltoken)

In [ ]:
df["clean_narrative"]=df["narrative"].apply(cleantext)

In [ ]:
x = df["clean_narrative"]
y = df["product"]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
sentlen = []
for sent in df['clean_narrative']:
  sentlen.append(len(word_tokenize(sent)))

df['Sentlen']=sentlen
df

In [ ]:
max(sentlen)

In [ ]:
np.quantile(sentlen,0.95)

In [ ]:
max_len =np.quantile(sentlen,0.95)

In [ ]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(xtrain)
tok.index_word

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

In [ ]:
seqtrain=tok.texts_to_sequences(xtrain)
seqtrain

In [ ]:
seqmattrain=sequence.pad_sequences(seqtrain,maxlen=int(max_len))
seqmattrain

In [ ]:
seqtest=tok.texts_to_sequences(xtest)
seqmattest=sequence.pad_sequences(seqtest,maxlen=int(max_len))

In [ ]:
vocab_len

In [ ]:
#model = Sequential()

##model.add(LSTM(units=10, activation="tanh"))
#model.add(Dense(units=10, activation="relu"))
##model.add(Dropout(0.2))

#model.add(Dense(units=5, activation="softmax"))

#model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
#model.fit(seqmattrain, ytrain, batch_size=1000, epochs=30)

#ypred = model.predict(seqmattest)
#ypred=ypred.argmax(axis=1)
#ypred

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(ytest, ypred))

In [ ]:
#msg=input('please eneter your msg :-     ')
#seqmsg = tok.texts_to_sequences([msg])
#seqmatmsg = sequence.pad_sequences(seqmsg, maxlen=240)
#ypred1 = model.predict(seqmatmsg)
#ypred1=ypred1.argmax()

#if ypred1==0:
 # print('credit_card')
#elif ypred1==1:
 # print('credit_reporting')
#elif ypred1==2:
 # print('debt_collection')
#elif ypred1==3:
 # print('mortgages_and_loans')
#elif ypred1==4:
 #   print('retail_banking')

please eneter your msg :-     I have noticed fraudulent activity on my credit card and request immediate investigation and resolution.
1/1 [==============================] - 0s 35ms/step
credit_card


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the complaints
tokenized_complaints = [tokenizer.encode(complaint, add_special_tokens=True) for complaint in complaints]

# Pad sequences to a fixed length
max_length = max(len(complaint) for complaint in tokenized_complaints)
padded_complaints = tf.keras.preprocessing.sequence.pad_sequences(tokenized_complaints, maxlen=max_length, padding='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_complaints, labels, test_size=0.2, random_state=42)

# Load the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze the BERT layers
bert_model.trainable = False

# Create the input layer
input_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# Get the BERT embeddings
bert_embeddings = bert_model(input_layer)[0]

# Apply pooling operation
pooled_output = tf.keras.layers.GlobalMaxPooling1D()(bert_embeddings)

# Add a dense layer and output layer
dense_layer = tf.keras.layers.Dense(64, activation='relu')(pooled_output)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(dense_layer)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(X_test)
